In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas
from dash import dcc
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sqlalchemy import create_engine

In [3]:
# engine = create_engine('postgresql://pi:pi@192.168.167.94:5432/Pergola')
engine = create_engine('postgresql://pi:pi@192.168.43.37:5432/Pergola')
df = pandas.read_sql(con=engine, sql='SELECT * FROM public."Juntek"')
df.pop("id")
df.to_csv("JuntekSQL.csv", index=False)

OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: SELECT * FROM public."Juntek"]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
df = pandas.read_csv("JuntekSQL.csv")
df["voltage"] = df["voltage"]/100
df["current"] = df["current"]/100
df["power"] = df["voltage"]*df["current"]
df["charged"] = df["charged"]/100
df["charging"] = df["power"].clip(lower=0)
df["discharging"] = df["power"].clip(upper=0)
df


In [ ]:
green = "rgb(18, 94, 3)"
green_light = "rgb(52, 130, 38)"
red = 'rgba(255, 0, 0, 0.5)'
orange = 'rgb(237, 159, 43)'
blue = "rgb(4, 72, 207)"

fig = go.Figure()

mask = df["power"] >= 0
mask2 = df["power"] <= 0
fig.add_trace(go.Scatter(x=df["time"], y=df["charging"], name="charging", fill='tozeroy', fillcolor=green, line=dict(color=green)))
fig.add_trace(go.Scatter(x=df["time"], y=df["discharging"], name="discharging", fill='tozeroy', fillcolor=red, line=dict(color=red)))
fig.add_trace(go.Scatter(x=df["time"], y=df["charged"], name="charged", yaxis="y2", line=dict(color=orange)))
fig.add_trace(go.Scatter(x=df["time"], y=df["voltage"], name="voltage", yaxis="y3", line=dict(color=blue)))
#fig.add_trace(go.Scatter(x=df["time"], y=[400000, 500000, 600000], name="yaxis4 data", yaxis="y4"))

# Create axis objects
fig.update_layout(xaxis=dict(domain=[0.0, 0.95], showgrid=False), 
                  yaxis=dict(titlefont=dict(color=green), tickfont=dict(color=green), range=[-200, 1000], showgrid=False),
                  yaxis2=dict(titlefont=dict(color=orange), tickfont=dict(color=orange), range=[-26, 130], tickvals = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                              anchor="free", overlaying="y", side="right", position=0.97, showgrid=False),
                  yaxis3=dict(titlefont=dict(color=blue), tickfont=dict(color=blue), range=[0, 29.5], tickvals = [22, 23, 24, 25, 26, 27, 28, 29],
                              anchor="x",  overlaying="y", side="right", showgrid=False),
                 )
                  #yaxis4=dict(title="yaxis4 title", titlefont=dict(color="#9467bd"), tickfont=dict(color="#9467bd"), anchor="free", overlaying="y",side="right", position=0.85))
fig.update_layout(template="plotly_dark")
# Update layout properties
fig.update_layout(title_text="",height=1000)

fig.show()